In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_pricing_decisions = pd.read_csv('train_prices_decisions.csv')
test_user_info = pd.read_csv('test_user_info.csv')

In [3]:
train_pricing_decisions.head()

,user_index,Covariate1,Covariate2,Covariate3,price_item_0,price_item_1,item_bought
0,0,4.319065,4.917636,6.054873,1.659785,3.598304,0
1,1,6.375840,7.228608,1.324521,13.120469,2.578281,1
2,2,1.524010,0.393772,4.873626,27.145855,4.538328,-1
3,3,1.243759,1.651083,2.268083,5.275166,14.818790,0
4,4,2.510326,8.265984,4.602699,12.681638,6.809336,0


In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, precision_recall_fscore_support

In [43]:
def calc_confusion_matrix(pred_label, test_label):
    true_label = test_label
    predicted_label = pred_label
    print("Confusion Matrix :")
    print(confusion_matrix(true_label, predicted_label))
    print("Classification Report :")
    print(classification_report(true_label, predicted_label, digits=4))
    print('Weighted FScore: \n ', precision_recall_fscore_support(true_label, predicted_label, average='weighted'))

In [17]:
def fit_logistic_regression_demand_with_covariates(df):
    # training
    model = LogisticRegression(random_state=1,solver='liblinear')
    model.fit(df[['Covariate1','Covariate2','Covariate3','price_item_0','price_item_1']].values, df['item_bought'].values)
    return model

In [40]:
def get_prediction_logistic(fitted_model, price, covariates):
    x = np.hstack((covariates, price))
    return fitted_model.predict_proba(x)

In [41]:
# LR Model
lr_model = fit_logistic_regression_demand_with_covariates(train_pricing_decisions)

In [48]:
# Confusion Matrix of the Training Set
price = train_pricing_decisions[['price_item_0','price_item_1']].values
covariates = train_pricing_decisions[['Covariate1','Covariate2','Covariate3']].values
train_pred = get_prediction_logistic(lr_model, price, covariates)
train_pred = np.argmax(train_pred, axis=1)
calc_confusion_matrix(train_pred, train_pricing_decisions['item_bought'].values+1)

Confusion Matrix :
[[7817  794  750]
 [ 581 9240  975]
 [ 788  856 8199]]
Classification Report :
              precision    recall  f1-score   support

           0     0.8510    0.8351    0.8429      9361
           1     0.8485    0.8559    0.8522     10796
           2     0.8262    0.8330    0.8296      9843

    accuracy                         0.8419     30000
   macro avg     0.8419    0.8413    0.8416     30000
weighted avg     0.8419    0.8419    0.8419     30000

Weighted FScore: 
  (0.841941382665533, 0.8418666666666667, 0.8418702735316798, None)


In [72]:
# Revenue = price(A) * demand(A) + price(B) * demand(B)
def get_revenue_maximizing_price_and_revenue(price_options, demand_predictions):
    price = [0,0]
    revenue = 0
    tmp = 0
    for i in range(len(price_options)):
        tmp = price_options[i][0] * demand_predictions[i][1] + price_options[i][1] * demand_predictions[i][2]
        if tmp > revenue:
            price[0] = price_options[i][0]
            price[1] = price_options[i][1]
            revenue = tmp
    return price, revenue

In [91]:
# Build a price set
priceSet = np.array([[1,2],[3,4],[5,6],[7,8],[9,10],[100,200]])
covariates = np.array([[4.319065, 4.917636, 6.054873]])
covariates = np.tile(covariates, (len(priceSet),1))
demand_predictions = get_prediction_logistic(lr_model, priceSet, covariates)

In [92]:
demand_predictions

array([[1.84138339e-02, 8.36013419e-01, 1.45572747e-01],
       [3.10068700e-02, 8.73914538e-01, 9.50785922e-02],
       [5.20893834e-02, 8.87488148e-01, 6.04224689e-02],
       [8.85986763e-02, 8.72894442e-01, 3.85068815e-02],
       [1.53811146e-01, 8.21067701e-01, 2.51211532e-02],
       [9.99999997e-01, 2.53495879e-09, 2.51774495e-31]])

In [93]:
opt_price, opt_revenue = get_revenue_maximizing_price_and_revenue(priceSet, demand_predictions)

In [94]:
print(opt_price)
print(opt_revenue)

[9, 10]
7.6408208388106615
